In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df=pd.read_csv('diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [5]:
x=df.drop(columns=['Outcome'])
y=df['Outcome']

In [6]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=.2)


In [7]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [8]:
# scaling the data 

In [9]:
from sklearn.preprocessing import  StandardScaler
scale=StandardScaler()
xtrain=scale.fit_transform(xtrain)
xtest=scale.fit_transform(xtest)

# Model without keras tuner

In [12]:
temp_model=Sequential()
temp_model.add(Dense(256,activation='relu',input_dim=8))
temp_model.add(Dense(128,activation='relu'))
temp_model.add(Dense(1,activation='sigmoid'))
temp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               2304      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 35329 (138.00 KB)
Trainable params: 35329 (138.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
temp_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [16]:
temp_model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=50)

Epoch 1/50
20/20 [==============================] - 2s 25ms/step - loss: 0.5792 - accuracy: 0.6938 - val_loss: 0.4737 - val_accuracy: 0.7792
Epoch 2/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4735 - accuracy: 0.7736 - val_loss: 0.4415 - val_accuracy: 0.7662
Epoch 3/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4562 - accuracy: 0.7752 - val_loss: 0.4436 - val_accuracy: 0.7727
Epoch 4/50
20/20 [==============================] - 0s 9ms/step - loss: 0.4477 - accuracy: 0.7915 - val_loss: 0.4465 - val_accuracy: 0.8052
Epoch 5/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4378 - accuracy: 0.7883 - val_loss: 0.4496 - val_accuracy: 0.7857
Epoch 6/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4382 - accuracy: 0.7883 - val_loss: 0.4481 - val_accuracy: 0.8052
Epoch 7/50
20/20 [==============================] - 0s 6ms/step - loss: 0.4254 - accuracy: 0.7980 - val_loss: 0.4485 - val_accuracy: 0.7987
Epoch 8/50
20/20 [=

So problems in this model is we dont able to know:

- how many layers we have to add

- how many node a particular layer have

- which loss function is best for each layer

- which activation function is suitable for every layer

- how many epoch the model have to train 

Here the keras tuner is came into the picture

# Model with keras tuner

In [17]:
import kerastuner as kt

C:\Users\Asus\AppData\Local\Temp\ipykernel_17404\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


there are 3 steps 

- first is you have to make a function named "build_model(hp)" // where hp is the object(hyperparameter)

- second is you have to make "tuner object" of a class RandomSearch()

- you have to call the "search()" function using tuner object

## how to tune the appropriate optimizer

- step1: build_model()

In [ ]:
def build_model(hp):
    pass